## Data Loading

In [2]:
import pandas as pd
df = pd.read_csv(r"C:\Users\jagan\OneDrive\Desktop\top_30_mutual_funds.csv")


In [2]:
print(df.describe())

           min_sip   min_lumpsum  expense_ratio  fund_size_cr  fund_age_yr  \
count   814.000000    814.000000     814.000000    814.000000   814.000000   
mean    528.500000   3047.468059       0.713231   3812.854214     8.319410   
std     369.184517   2514.619513       0.482161   7181.479475     2.640622   
min       0.000000      0.000000       0.000000      2.380000     1.000000   
25%     150.000000    500.000000       0.310000    175.250000     6.000000   
50%     500.000000   5000.000000       0.615000    940.500000    10.000000   
75%    1000.000000   5000.000000       1.030000   3819.000000    10.000000   
max    2000.000000  25000.000000       2.590000  57052.000000    17.000000   

       risk_level      rating  returns_1yr  returns_3yr  returns_5yr  
count  814.000000  814.000000   814.000000   793.000000   647.000000  
mean     4.400491    2.641278     3.921376    18.525347     9.490726  
std      1.814741    1.463476     6.675502    12.108476     3.714015  
min      1.00

In [3]:
df['category'].unique()

array(['Equity', 'Hybrid', 'Other', 'Debt', 'Solution Oriented'],
      dtype=object)

In [4]:
df.isnull().sum()

scheme_name        0
min_sip            0
min_lumpsum        0
expense_ratio      0
fund_size_cr       0
fund_age_yr        0
fund_manager       0
sortino            0
alpha              0
sd                 0
beta               0
sharpe             0
risk_level         0
amc_name           0
rating             0
category           0
sub_category       0
returns_1yr        0
returns_3yr       21
returns_5yr      167
dtype: int64

## Data Cleaning 

In [5]:
# Replace missing values in returns_3yr and returns_5yr
mean_returns_3yr = df['returns_3yr'].mean()
mean_returns_5yr = df['returns_5yr'].mean()
df['returns_3yr'].fillna(mean_returns_3yr, inplace=True)
df['returns_5yr'].fillna(mean_returns_5yr, inplace=True)
print(df.isnull().sum())

scheme_name      0
min_sip          0
min_lumpsum      0
expense_ratio    0
fund_size_cr     0
fund_age_yr      0
fund_manager     0
sortino          0
alpha            0
sd               0
beta             0
sharpe           0
risk_level       0
amc_name         0
rating           0
category         0
sub_category     0
returns_1yr      0
returns_3yr      0
returns_5yr      0
dtype: int64


## Data Normalization

In [6]:
from sklearn.preprocessing import MinMaxScaler

columns_to_normalize = ['expense_ratio', 'returns_1yr', 'returns_3yr', 'returns_5yr', 
                        'sharpe', 'sortino', 'alpha', 'beta']
df[columns_to_normalize] = df[columns_to_normalize].replace('-', pd.NA).apply(pd.to_numeric)

scaler = MinMaxScaler()
df_normalized = pd.DataFrame(scaler.fit_transform(df[columns_to_normalize]), columns=columns_to_normalize)

# Adjust metrics where lower is better
df_normalized['expense_ratio'] = 1 - df_normalized['expense_ratio']
df_normalized['beta'] = 1 - df_normalized['beta']

ModuleNotFoundError: No module named 'sklearn'

## Composite Scoring

In [7]:
weights = {
    'expense_ratio': 0.2,
    'returns_1yr': 0.15,
    'returns_3yr': 0.15,
    'returns_5yr': 0.15,
    'sharpe': 0.1,
    'sortino': 0.1,
    'alpha': 0.1,
    'beta': 0.05
}

df_normalized['composite_score'] = sum(
    df_normalized[col] * weight for col, weight in weights.items()
)
df['composite_score'] = df_normalized['composite_score']


## Ranking Fund

In [8]:
df['rank'] = df['composite_score'].rank(ascending=False)
df_sorted = df.sort_values(by='rank')

## Exporting Result

In [9]:
df_top_30 = df_sorted.head(30)
df_top_30.to_excel('top_30_mutual_funds.xlsx', index=False)
print("Exported top 30 mutual funds to 'top_30_mutual_funds.xlsx'.")

Exported top 30 mutual funds to 'top_30_mutual_funds.xlsx'.
